In [3]:
# python --version
!pip install --upgrade luxai_s2
!pip install importlib-metadata==4.13.0
!pip install --upgrade moviepy # needed to render videos of episodes

  Using cached luxai_s2-2.1.9-py3-none-any.whl (63 kB)
  Using cached PettingZoo-1.22.3-py3-none-any.whl (816 kB)
  Using cached termcolor-2.2.0-py3-none-any.whl (6.6 kB)
     |████████████████████████████████| 12.8 MB 540 kB/s eta 0:00:01
  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Using cached vec_noise-1.1.4.zip (134 kB)
  Using cached gymnasium-0.27.1-py3-none-any.whl (883 kB)
  Using cached gymnasium_notices-0.0.1-py3-none-any.whl (2.8 kB)
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached jax_jumpy-0.2.0-py3-none-any.whl (11 kB)
     |████████████████████████████████| 19.8 MB 359 kB/s eta 0:00:01     |████████████████▎               | 10.1 MB 268 kB/s eta 0:00:37
     |████████████████████████████████| 17.6 MB 331 kB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616827 sha256=37d86193c9f5002dca02b7374907d4aae1c636a25a6bcb69c52b3aecfda03b40
  Stored in directory: /Users/itong1900/Library/Caches/pip/wheels/27/6d/b3/a

In [14]:
from luxai_s2 import LuxAI_S2
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(threshold=np.inf)

AttributeError: 'EntryPoints' object has no attribute 'get'

In [11]:
env = LuxAI_S2() # create the environment object
obs = env.reset(seed=256) # resets an environment with a seed

NameError: name 'LuxAI_S2' is not defined

## here is what a typical map looks like

There are 2 types - mountains (on the left) and caves (on the right)

The color coding is:
* dark red for high rubble tiles
* light red for low rubble tiles
* blue for ice
* yellow for ore

You want your factory to be near ice and ore (in order to get resourses efficiently and safely)
and you want to be orthogonally connected to a large low-rubble zone (so that you can actually grow lichen). Ideally you also want to block the opponent's access to low rubble zones.

### before we continue, try looking at each map and consider, what are the best starting locations?

In [ ]:
mountain_obs = env.reset(seed=420)
mountain_map = env.render("rgb_array", width=48, height=48).transpose(1,0,2)
cave_obs = env.reset(seed=1026)
cave_map = env.render("rgb_array", width=48, height=48).transpose(1,0,2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
im1 = ax1.imshow(mountain_map)
ax1.set_title("mountain")
im2 = ax2.imshow(cave_map)
ax2.set_title("cave")
plt.figure(dpi=150)
plt.show()

### Let's start with the ice. What are the best locations (near ice)?

In [ ]:
from scipy.ndimage import distance_transform_cdt

def manhattan_distance(binary_mask):
    # Get the distance map from every pixel to the nearest positive pixel
    distance_map = distance_transform_cdt(binary_mask, metric='taxicab')

    return distance_map


ice = cave_obs["player_0"]["board"]["ice"]
dist_ice = manhattan_distance(1-ice)


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
im1 = ax1.imshow(cave_map)
ax1.set_title('map')
im2 = ax2.imshow(ice)
ax2.set_title("ice")
im3 = ax3.imshow(dist_ice)
ax3.set_title("distance to the closest ice")
fig.colorbar(im3, ax=ax3)
plt.figure(dpi=150)
plt.show()

### We have some candidate locations!
Let's also consider the distance to the nearest ore, and whether a coordinate is a valid spawn location.

In [ ]:
ice = cave_obs["player_0"]["board"]["ice"]
ore = cave_obs["player_0"]["board"]["ore"]
rubble = cave_obs["player_0"]["board"]["rubble"]

dist_ice = manhattan_distance(1 - ice)
dist_ice = np.max(dist_ice) - dist_ice
dist_ore = manhattan_distance(1 - ore)
dist_ore = np.max(dist_ore) - dist_ore

score = dist_ice + dist_ore
valid_good_spawns = score * cave_obs["player_0"]["board"]["valid_spawns_mask"]

best_loc = np.argmax(valid_good_spawns)
x, y = np.unravel_index(best_loc, (48, 48))
print(f"best starting location by distance to the nearest ice/ore is {(x, y)}")


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

im1 = ax1.imshow(cave_map)
ax1.set_title('map')

im2 = ax2.imshow(score)
ax2.set_title('combined distance to ore and ice')
fig.colorbar(im2, ax=ax2)

im3 = ax3.imshow(valid_good_spawns, cmap='hot')
ax3.set_title('valid good spawns')
fig.colorbar(im3, ax=ax3)

plt.show()

This is already decent, it suggests a factory directly adjacent to both ice and ore. A good start.

### What if we prefer to have 2 ice sources near our factory? Or 2 ore sources?

Let's also calculate the distance to the *second* closest ice/ore source. And third, and fourth.

In [ ]:
from scipy.spatial import KDTree

def manhattan_dist_to_nth_closest(arr, n):
    if n == 1:
        distance_map = distance_transform_cdt(1-arr, metric='taxicab')
        return distance_map
    else:
        true_coords = np.transpose(np.nonzero(arr)) # get the coordinates of true values
        tree = KDTree(true_coords) # build a KDTree
        dist, _ = tree.query(np.transpose(np.nonzero(~arr)), k=n, p=1) # query the nearest to nth closest distances using p=1 for Manhattan distance
        return np.reshape(dist[:, n-1], arr.shape) # reshape the result to match the input shape and add an extra dimension for the different closest distances

# this is the distance to the n-th closest ice, for each coordinate
ice_distances = [manhattan_dist_to_nth_closest(ice, i) for i in range(1,5)]

# this is the distance to the n-th closest ore, for each coordinate
ore_distances = [manhattan_dist_to_nth_closest(ore, i) for i in range(1,5)]

fig, axs = plt.subplots(2, 3, figsize=(15, 10))
axs[0, 0].imshow(cave_map)
axs[0, 0].set_title('map')
axs[0, 1].imshow(ice)
axs[0, 1].set_title('ice')
axs[0, 2].imshow(ice_distances[0], cmap='hot')
axs[0, 2].set_title('distance to closest ice')
axs[1, 0].imshow(ice_distances[1], cmap='hot')
axs[1, 0].set_title('distance to 2nd closest ice')
axs[1, 1].imshow(ice_distances[2], cmap='hot')
axs[1, 1].set_title('distance to 3rd closest ice')
axs[1, 2].imshow(ice_distances[3], cmap='hot')
axs[1, 2].set_title('distance to 4th closest ice')

### Let's combine everything so far!

One final consideration - we care more about the *first* source of ice, or ore, than for each subsequent one. 
So let's weigh them accordingly. 

We're also going to add overall weights for ice and ore (ore seems to be less important overall, at least for beginner agents).

Feel free to tweak all parameters and play around!

In [ ]:
ICE_WEIGHTS = np.array([1, 0.5, 0.33, 0.25]) 
weigthed_ice_dist = np.sum(np.array(ice_distances) * ICE_WEIGHTS[:, np.newaxis, np.newaxis], axis=0)

ORE_WEIGHTS = np.array([1, 0.5, 0.33, 0.25])
weigthed_ore_dist = np.sum(np.array(ore_distances) * ORE_WEIGHTS[:, np.newaxis, np.newaxis], axis=0)

ICE_PREFERENCE = 3 # if you want to make ore more important, change to 0.3 for example

combined_resource_score = (weigthed_ice_dist * ICE_PREFERENCE + weigthed_ore_dist)
combined_resource_score = (np.max(combined_resource_score) - combined_resource_score) * cave_obs["player_0"]["board"]["valid_spawns_mask"]


best_loc = np.argmax(combined_resource_score)
x, y = np.unravel_index(best_loc, (48, 48))
print(f"best starting location according to weighted distances to ore and ice is {(x, y)}")

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(18, 5))

im1 = ax1.imshow(cave_map)
ax1.set_title('map')

im2 = ax2.imshow(weigthed_ice_dist)
ax2.set_title('weigthed_ice_dist')

im3 = ax3.imshow(weigthed_ore_dist)
ax3.set_title('weigthed_ore_dist')

im4 = ax4.imshow(combined_resource_score)
ax4.set_title('weighted distances to resources score')

plt.show()                    

#### Time to include low rubble areas in the calculation.

In order to grow lichen, we need to clean cells, directly connected to our factory, from all rubble. 
So, the best strategy is to find low rubble zones, and place our factory right next to them. 

Below, we calculate how many low-rubble cells there are near each possible factory location. 

We have 2 important parameters here:
- maximum_depth, which controls how far away we search for low-rubble areas
- exponent, which controls how much we care for more distant areas

Here is a visualization of what locations are good for this map, focusing only on the rubble. Again, feel free to tweak the parameters and test.

In [ ]:
low_rubble = (rubble<25)

def count_region_cells(array, start, min_dist=2, max_dist=np.inf, exponent=1):
    
    def dfs(array, loc):
        distance_from_start = abs(loc[0]-start[0]) + abs(loc[1]-start[1])
        if not (0<=loc[0]<array.shape[0] and 0<=loc[1]<array.shape[1]):   # check to see if we're still inside the map
            return 0
        if (not array[loc]) or visited[loc]:     # we're only interested in low rubble, not visited yet cells
            return 0
        if not (min_dist <= distance_from_start <= max_dist):      
            return 0
        
        visited[loc] = True

        count = 1.0 * exponent**distance_from_start
        count += dfs(array, (loc[0]-1, loc[1]))
        count += dfs(array, (loc[0]+1, loc[1]))
        count += dfs(array, (loc[0], loc[1]-1))
        count += dfs(array, (loc[0], loc[1]+1))

        return count

    visited = np.zeros_like(array, dtype=bool)
    return dfs(array, start)

low_rubble_scores = np.zeros_like(low_rubble, dtype=float)

for i in range(low_rubble.shape[0]):
    for j in range(low_rubble.shape[1]):
        low_rubble_scores[i,j] = count_region_cells(low_rubble, (i,j), min_dist=0, max_dist=8, exponent=0.9)



plt.figure(dpi=150)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

im1 = ax1.imshow(cave_map)
ax1.set_title('map')

im2 = ax2.imshow(low_rubble_scores)
ax2.set_title('low rubble nearby')
fig.colorbar(im2, ax=ax2)

### Finally, let's combine everything so far

In [ ]:
overall_score = (low_rubble_scores*2 + combined_resource_score ) * cave_obs["player_0"]["board"]["valid_spawns_mask"]

best_loc = np.argmax(overall_score)
x, y = np.unravel_index(best_loc, (48, 48))
print(f"best starting location according to the combined resource and rubble metrics is {(x, y)}")

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(23, 5))

im1 = ax1.imshow(cave_map)
ax1.set_title('map')

im2 = ax2.imshow(low_rubble_scores)
ax2.set_title('low rubble nearby')
fig.colorbar(im2, ax=ax2)

im3 = ax3.imshow(combined_resource_score)
ax3.set_title('weighted distances to resources')
fig.colorbar(im3, ax=ax3)

im4 = ax4.imshow(overall_score)
ax4.set_title('tadaaaaa')
fig.colorbar(im4, ax=ax4)